## Мониторим, что алгоритм не меняет поведение #93

При рефакторинге смотрим, чтобы зафиксированные результаты не менялись. Это означает, что всё работает так же, как прежде и мы ничего не сломали (но это не точно)

In [3]:
%reload_ext autoreload
%autoreload 2

In [7]:
from app.helper import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig


from concurrent.futures import ThreadPoolExecutor, as_completed

TICKER = 'RNFT'

test_configs = [
    (RunConfig(
        ticker=TICKER,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=2,

        majority_trade=True,
        pretest_period=pretest_period,  
        pretest_type=pretest_type,

        threshold_buy_steps=threshold_b_n,
        threshold_sell_steps=threshold_s_n,
        stop_up_p=stop_up_p,
        stop_down_p=stop_down_p,

        step_size=step_size,
        step_set_orders_cnt=step_cnt,
    ))
    for max_shares in [3, 4]
    for base_shares in [0]
    for threshold_b_n in [0]
    for threshold_s_n in [0]
    for step_size in [1.6]
    for step_cnt in [3]
    for pretest_period in [6]  # range(2, 7)
    for pretest_type in [RunConfig.PRETEST_PRE]
    for stop_up_p in [0]
    for stop_down_p in [0]
]


def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config)
    return test_alg.test(
        last_test_date='2024-08-20',
        test_days_num=30,  # 365,
        shares_count=0,
    )

results = []
LocalCache.clear()
progress = TaskProgress(len(test_configs))

with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in test_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: float(x['profit_p']), reverse=True)

print()
for item in sorted_results:
    print(item)

Запуск в 14:40
Закончено в 14:40, количество 2, длительность 0:00:04                                               

{'profit': 95.8, 'profit_p': 3.58, 'profit_p_avg': 0.12, 'config': RNFT+ 4/pre6:0/3 x l2 x 1.6¤ , 'last_conf': RNFT+ 4/pre6:0/3 x l2 x 1.6¤ , 'op': 204}
{'profit': 40.5, 'profit_p': 2.02, 'profit_p_avg': 0.07, 'config': RNFT+ 3/pre6:0/3 x l2 x 1.6¤ , 'last_conf': RNFT+ 3/pre6:0/3 x l2 x 1.6¤ , 'op': 140}


### Запуск старого алгоритма
last_test_date='2024-08-20',
test_days_num=30,  # 365,

{'profit': 95.8, 'profit_p': 3.58, 'profit_p_avg': 0.12, 'config': RNFT+ 4/pre6:0/3 x l2 x 1.6¤ , 'op': 204}

{'profit': 40.5, 'profit_p': 2.02, 'profit_p_avg': 0.07, 'config': RNFT+ 3/pre6:0/3 x l2 x 1.6¤ , 'op': 140}
